# Async and Await Primer

A quick primer on `async`/`await`. Async and await are relatively new features in Python which allow **concurent** programming. They won't make your code magically faster, but may make your code easier to read, maintain and reason about. 
You will likely hear the terms event-loop, coroutines and many other ones, they will make sens in time. 

The key thing to remember is that 
 - async-functions can call both sync and async functions.
 - sync functions can only call sync. 
 - You _must_ always `await <async functions>`


## Event loop

It's like the "One ring", there shoudl be only one. IPython (and Jupyter) usually already run one.

### Bad news
If you need to run any code that need to create and manage an event-loop, consult the docs. 
Typically you can't run a tornado app inside jupyter.

### Good news

If you don't know/don't care, all is already setup for you. 



## Example

Let's deactivate enventloop integration and try what is (usually invalid Python)

In [1]:
%autoawait False

In [2]:
from asyncio import sleep

In [3]:
# does not sleep, need to be awaited
print('before sleep')
sleep(5)
print('after sleep')

before sleep
after sleep


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: coroutine 'sleep' was never awaited
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
await sleep(5)

SyntaxError: 'await' outside function (<ipython-input-4-920ac5bc3075>, line 1)

In [5]:
def f():
    await sleep(5)
f()

SyntaxError: 'await' outside async function (<ipython-input-5-8c14fef35f4f>, line 2)

In [6]:
async def f():
    print('before...')
    await sleep(5)
    print('after')
### does not call f
f()

<coroutine object f at 0x000001EDA0E37048>

In [7]:
await f()

SyntaxError: 'await' outside function (<ipython-input-7-9fe41d264da1>, line 1)

... back to step beginning. 

## Autoawait

Autoawait will _attempt_ to detect async code and run it for you. There are of course limitations (bug report welcome)

In [8]:
%autoawait True

You will note that any line that start with `%` is invalid Python and are IPython specific syntax. Those are call magics (line-magics with a single `%` sign, cell magics with a double `%%` sign)

In [9]:
print('before')
await sleep(5)
print('after')

before
after


Top level await is now valid syntax. 

In [10]:
tpl = 'https://anapioficeandfire.com/api/characters/{}'

In [11]:
%%time

results = []
for i in range(1,50):
    import requests
    print('.', end='')
    r = requests.get(tpl.format(i)).json()['aliases']
    print('x', end='')
    results.append(r)
    
for r in results:
    print(r)

.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x['The Daughter of the Dusk']
['Hodor']
['Lamprey']
['The Merling Queen']
['Old Crackbones']
['The Poetess']
['Porridge']
['Quickfinger']
["the Sailor's Wife"]
['The Veiled Lady']
['The waif']
['Balon the Brave', 'Balon the Blessed', 'Balon the Twice Crowned', 'Balon the Widowmaker', 'The Kraken King']
['']
['']
['The High Sparrow']
['The Little Queen', 'The Little Rose', 'Maid Margaery']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['The Lion of Lannister', 'The Old Lion', 'The Great Lion of the Rock']
['']
['']
['']
['']
['']
['Addam of Hull']
['The Bastard of Cornfield']
['']
['Jinglebell']
['Aegon Bloodborn']
['Aegon the Conqueror', 'Aegon the Dragon', 'Aegon the Dragonlord']
['Aegon the Elder', 'Aegon the Usurper']
['Aegon the Younger', 'Aegon the Dragonbane', 'Aegon the Unlucky', 'The Broken King']
['Aegon the Unworthy']
['Young Griff']
['']
['']
['']
['Aegon the Unlikely', 'Aego

# Moving to asynchronous

Nothing is perfect; if you get RuntimeErrors with asyncio, you may need to restart your kernel. More during my colleagues aiohttp tutorial this Afternoon

In [13]:
import aiohttp

In [14]:
async with aiohttp.ClientSession() as session:
    response = await session.get(tpl.format(583))
    json = await response.json()
    print(json['aliases'])

['Lord Snow', "Ned Stark's Bastard", 'The Snow of Winterfell', 'The Crow-Come-Over', "The 998th Lord Commander of the Night's Watch", 'The Bastard of Winterfell', 'The Black Bastard of the Wall', 'Lord Crow']


In [15]:
async def get_char(i, session):
        print('.', end='')
        response = await session.get(tpl.format(i))
        json = await response.json()
        print('x', end='')
        return json['aliases']

In [16]:
async with aiohttp.ClientSession() as s:
    print(await get_char(1303, s))

.x['Dany', 'Daenerys Stormborn', 'The Unburnt', 'Mother of Dragons', 'Mother', 'Mhysa', 'The Silver Queen', 'Silver Lady', 'Dragonmother', 'The Dragon Queen', "The Mad King's daughter"]


In [17]:
tasks = []

In [18]:
import asyncio
async with aiohttp.ClientSession() as session:
    # start 
    for i in range(1,50):
        task = asyncio.ensure_future(get_char(i, session))
        tasks.append(task)
    results = await asyncio.gather(*tasks)
    for r in results:
        print(r)

.................................................xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx['The Daughter of the Dusk']
['Hodor']
['Lamprey']
['The Merling Queen']
['Old Crackbones']
['The Poetess']
['Porridge']
['Quickfinger']
["the Sailor's Wife"]
['The Veiled Lady']
['The waif']
['Balon the Brave', 'Balon the Blessed', 'Balon the Twice Crowned', 'Balon the Widowmaker', 'The Kraken King']
['']
['']
['The High Sparrow']
['The Little Queen', 'The Little Rose', 'Maid Margaery']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['The Lion of Lannister', 'The Old Lion', 'The Great Lion of the Rock']
['']
['']
['']
['']
['']
['Addam of Hull']
['The Bastard of Cornfield']
['']
['Jinglebell']
['Aegon Bloodborn']
['Aegon the Conqueror', 'Aegon the Dragon', 'Aegon the Dragonlord']
['Aegon the Elder', 'Aegon the Usurper']
['Aegon the Younger', 'Aegon the Dragonbane', 'Aegon the Unlucky', 'The Broken King']
['Aegon the Unworthy']
['Young Griff']
['']
['']
['']
['Aegon the Unlikely', 'Aego

## Advance Autoawait usage, Exercise

Find the documentation for autoawait, and try to make it work with another asynchronous library. For exampe try to ply with [`trio`](https://trio.readthedocs.io/en/latest/), using `trio.sleep`  and `trio.open_nursery` to get several concurent task running, pritning different message regularly and at random intervals. What happen if you use `time.sleep()` instead of `trio.sleep()` ? What hapen if you use `asyncio.sleep()` ?

In [ ]:
import trio

In [ ]:
async def every(n, message):
    for i in range(30):
        await trio.sleep(n)
        print(message)

In [ ]:
%autoawait trio

In [ ]:
async with trio.open_nursery() as nursery:
    nursery.start_soon(every, 1, 'Plic')
    nursery.start_soon(every, 2, 'Ploc')
    